In [10]:
import numpy as np
import scipy.stats as stats

In [11]:
import lightgbm as lgbm

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [13]:
from sklearn.model_selection import RandomizedSearchCV

In [14]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [15]:
labels = pd.read_csv('labels_training_set.csv')

In [16]:
processed = pd.read_csv('features_10.csv')

In [17]:
for col in processed.columns:
    if processed[col].dtype.name == 'object':
        processed[col] = processed[col].astype('category')

In [18]:
processed = processed.set_index(processed.columns[0])
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,4GB_vio,512MB_compro,512MB_vio,region_sin_unknown,pais_sin_unknown,precio_del_cel_mas_visto,promedio_precio_celulares_vistos,promedio_precio_celulares_comprados,precio_mas_caro_que_vio,precio_mas_barato_que_vio
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,05-17,05-17,6,1,...,False,False,False,Sao Paulo,Brazil,NaN,NaN,0.0,NaN,NaN
00091926,viewed product,iPhone 6S,Computer,0,34,372,05-03,05-31,448,28,...,False,False,False,Rio Grande do Sul,Brazil,1679.0,1527.091398,0.0,4649.0,100.0
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,03-26,03-26,10,1,...,False,False,False,Minas Gerais,Brazil,1679.0,1342.333333,0.0,1679.0,849.0
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,05-17,05-26,206,10,...,True,False,False,Sao Paulo,Brazil,369.0,479.366013,379.0,2229.0,250.0
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,05-29,05-29,17,1,...,False,False,False,Minas Gerais,Brazil,2229.0,2229.000000,0.0,2229.0,2229.0


In [19]:
predict_features = kaggle.set_index('person').join(processed)

In [20]:
predict_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,4GB_vio,512MB_compro,512MB_vio,region_sin_unknown,pais_sin_unknown,precio_del_cel_mas_visto,promedio_precio_celulares_vistos,promedio_precio_celulares_comprados,precio_mas_caro_que_vio,precio_mas_barato_que_vio
person,,,,,,,,,,,,,,,,,,,,,
4886f805,viewed product,Samsung Galaxy J7 Prime,Smartphone,0,1,4,05-18,05-18,9,1,...,False,False,False,Rio de Janeiro,Brazil,649.0,649.000000,0.0,649.0,649.0
0297fc1e,viewed product,iPhone 6,Smartphone,0,95,404,01-09,05-28,567,139,...,False,False,False,Rio de Janeiro,Brazil,1499.0,1641.153465,0.0,4649.0,390.0
2d681dd8,viewed product,iPhone 7,Computer,0,2,13,05-18,05-27,26,10,...,False,False,False,Sao Paulo,Brazil,2229.0,1334.384615,0.0,2229.0,439.0
cccea85e,viewed product,Motorola Moto G4 Plus,Computer,0,22,739,05-07,05-31,836,24,...,False,False,False,Sao Paulo,Brazil,619.0,870.018945,0.0,2229.0,145.0
4c8a8b93,viewed product,iPhone 6,Smartphone,0,20,177,05-18,05-22,257,5,...,False,False,False,Sao Paulo,Brazil,1499.0,1494.310734,0.0,4649.0,549.0


In [21]:
training_features = labels.set_index('person').join(processed)
training_labels = training_features['label']

training_features = training_features.drop('label',axis=1)

In [22]:
training_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,4GB_vio,512MB_compro,512MB_vio,region_sin_unknown,pais_sin_unknown,precio_del_cel_mas_visto,promedio_precio_celulares_vistos,promedio_precio_celulares_comprados,precio_mas_caro_que_vio,precio_mas_barato_que_vio
person,,,,,,,,,,,,,,,,,,,,,
0566e9c1,viewed product,iPhone 5s,Smartphone,1,17,23,05-22,05-31,68,9,...,False,False,False,Parana,Brazil,999.0,663.913043,459.0,999.0,269.0
6ec7ee77,visited site,Ninguno,Smartphone,0,1,0,05-28,05-28,2,1,...,False,False,False,Rio Grande do Norte,Brazil,NaN,NaN,0.0,NaN,NaN
abe7a2fb,viewed product,iPhone 6,Smartphone,0,22,31,02-10,05-29,96,109,...,False,False,True,Sao Paulo,Brazil,1499.0,1184.161290,0.0,1589.0,179.0
34728364,viewed product,Lenovo Vibe K5,Smartphone,0,4,24,05-18,05-27,37,9,...,True,False,False,Sao Paulo,Brazil,479.0,529.916667,0.0,849.0,60.0
87ed62de,viewed product,Samsung Galaxy J7,Computer,0,1,9,05-18,05-19,17,1,...,False,False,False,Santa Catarina,Brazil,549.0,549.000000,0.0,549.0,549.0


In [23]:
params = {
    'num_leaves':stats.randint(90,15000),
    'min_data_in_leaf':stats.randint(100,1000),
    'max_bin':stats.randint(70,300),
    'learning_rate':stats.uniform(0.001,1-0.001),
    'n_estimators':stats.randint(150,1200)
}
grid = RandomizedSearchCV(lgbm.LGBMClassifier(boosting_type='dart'),
                          param_distributions=params,
                          scoring='roc_auc',
                          cv=2,
                          verbose=1,
                          n_iter=150)

In [24]:
xtr,xte,ytr,yte=train_test_split(training_features,training_labels)

In [25]:
grid.fit(xtr,ytr)

Fitting 2 folds for each of 150 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 31.6min finished


RandomizedSearchCV(cv=2, error_score='raise',
          estimator=LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
          fit_params=None, iid=True, n_iter=150, n_jobs=1,
          param_distributions={'num_leaves': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002311F0B0E10>, 'min_data_in_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002311F0B0A20>, 'max_bin': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002311EAB2278>, 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002311EAB2F60>, 'n_estimators': <scipy.stats._distn_infrastructure.rv

In [26]:
grid.score(X=xte,y=yte)

0.8484914260907315

In [27]:
grid.best_params_

{'learning_rate': 0.046410351906179786,
 'max_bin': 265,
 'min_data_in_leaf': 388,
 'n_estimators': 202,
 'num_leaves': 11864}

## Prediccion

In [49]:
predictor = grid.best_estimator_

In [50]:
predictor.feature_importances_

array([  3, 299,   0, 150,  62, 253, 571, 152, 595, 378,   0,   0,  81,
         0, 398,  70,  51,  55,  78,  59, 121,   1,   0, 161, 185,  14,
         1, 192, 107,   0,  13,  79,   0,   0])

In [51]:
predictor.fit(training_features,training_labels)

LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.06748371705664348,
        max_bin=205, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_data_in_leaf=481, min_split_gain=0.0,
        n_estimators=187, n_jobs=-1, num_leaves=956, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [53]:
prediction_submit = predictor.predict_proba(predict_features)[:,1]

In [55]:
submit = kaggle.set_index('person')

In [56]:
submit['label'] = prediction_submit

In [57]:
submit.to_csv('submit.csv')